In [ ]:
!pip install sparselayer-tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, ReLU, BatchNormalization, Flatten, MaxPool2D

from sparselayer_tensorflow import SparseLayerConv2D, SparseLayerDense

## Test SparseDense and SparseConv2D

In [ ]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

In [ ]:
X = tf.keras.layers.Input(shape=(28, 28, 1))
x = SparseLayerConv2D(32, 0.5, (3,3), (1,1))(X)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)

x = SparseLayerConv2D(64, 0.3, (3,3), (1,1))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)

x = Flatten()(x)
y = SparseLayerDense(units=10, density=0.2, activation=tf.nn.softmax)(x)

model = tf.keras.models.Model(X, y)

In [ ]:
model.summary()

In [ ]:
# Hyperparameters
batch_size=256
epochs=30


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train[:10000],
    y_train[:10000],
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

In [ ]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=256, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))